# Step 02: Load Configuration File

This notebook loads and validates the Excel configuration file for the analysis cycle.

**Tasks:**
- Select and validate Excel configuration file
- Validate file format and required sheets
- Validate data completeness and versions
- Validate reference data existence
- Load configuration to database

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.configuration import (
    validate_configuration_file,
    load_configuration_file,
    read_configuration,
    get_base_portfolios,
    classify_groupings
)
from helpers.database import execute_query

## 1) Setup

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Validate_Config_File.ipynb')

# Display context
ux.header("Configuration Loading")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")

# Check for existing configurations
existing_configs = execute_query(
    "SELECT id, status, created_ts FROM irp_configuration WHERE cycle_id = (SELECT id FROM irp_cycle WHERE cycle_name = %s)",
    (context.cycle_name,)
)

if not existing_configs.empty:
    ux.warning("⚠ This cycle already has a configuration loaded")
    for idx, config in existing_configs.iterrows():
        ux.info(f"  - Config ID: {config['id']}, Status: {config['status']}, Created: {config['created_ts']}")
    ux.warning("Loading a new configuration will replace the existing one")
    print()

ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Select Configuration File

In [ ]:
# Get Excel configuration file
ux.subheader("Select Configuration File")

# Get the expected configuration directory
config_dir = context.cycle_directory / "files" / "configuration"
ux.info(f"Configuration directory: {config_dir}")

# Check if directory exists
if not config_dir.exists():
    ux.warning(f"Configuration directory does not exist: {config_dir}")
    ux.info("Creating directory...")
    config_dir.mkdir(parents=True, exist_ok=True)
    ux.success("✓ Directory created")

# List available Excel files in the directory
excel_files = sorted(list(config_dir.glob("*.xlsx")) + list(config_dir.glob("*.xls")))

if not excel_files:
    ux.error(f"✗ No Excel files found in configuration directory")
    ux.info(f"\nPlease place your configuration file in: {config_dir}")
    raise FileNotFoundError("No configuration files found in directory")

# Display available files
ux.info(f"\nFound {len(excel_files)} Excel file(s):")
print()
for i, f in enumerate(excel_files, 1):
    file_size = f.stat().st_size / 1024  # KB
    from datetime import datetime
    file_modified = datetime.fromtimestamp(f.stat().st_mtime).strftime('%Y-%m-%d %H:%M')
    ux.info(f"  [{i}] {f.name}")
    ux.info(f"      Size: {file_size:.2f} KB | Modified: {file_modified}")
    print()

# Get user selection
selection = ux.text_input(
    f"Select configuration file [1-{len(excel_files)}]:",
    default="1"
)

try:
    selected_index = int(selection) - 1
    if selected_index < 0 or selected_index >= len(excel_files):
        raise ValueError("Selection out of range")
    config_path = excel_files[selected_index]
except (ValueError, IndexError) as e:
    ux.error(f"✗ Invalid selection: {selection}")
    raise ValueError(f"Invalid file selection. Please enter a number between 1 and {len(excel_files)}")

# Validate file exists (should always be true at this point, but keeping as sanity check)
if not config_path.exists():
    ux.error(f"✗ File not found: {config_path.name}")
    ux.info(f"Full path checked: {config_path}")
    raise FileNotFoundError(f"Configuration file not found: {config_path.name}")

if not config_path.suffix.lower() in ['.xlsx', '.xls']:
    ux.error(f"✗ Invalid file format: {config_path.suffix}")
    raise ValueError("Configuration file must be an Excel file (.xlsx or .xls)")

# Display selected file information
ux.success(f"\n✓ Selected: {config_path.name}")
file_size = config_path.stat().st_size / 1024  # KB
file_modified = config_path.stat().st_mtime

from datetime import datetime
file_info = [
    ["File Name", config_path.name],
    ["File Path", str(config_path)],
    ["File Size", f"{file_size:.2f} KB"],
    ["Last Modified", datetime.fromtimestamp(file_modified).strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(file_info, headers=["Property", "Value"])

# Log file selection
step.log(f"Configuration file selected: {config_path.name}")

## 3) Validate Configuration File

In [ ]:
# Validate configuration file structure and content
ux.header("Configuration Validation")
ux.info("Validating configuration file structure and content...")

# Run validation (returns validation results)
validation_response = validate_configuration_file(str(config_path))

# Extract components from response
validation_passed = validation_response['validation_passed']
config_data = validation_response['configuration_data']
file_info = validation_response['file_info']

# Check if validation data is present
if '_validation' not in config_data:
    ux.error("✗ Validation results not found in configuration data")
    raise ValueError("Configuration validation failed to produce results")

validation_data = config_data['_validation']

# Display validation summary
ux.subheader("Validation Summary")

summary_rows = []

for sheet_name, result in validation_data.items():
    status = result.get('status', 'UNKNOWN')
    status_icon = "✓" if status == 'SUCCESS' else "✗"
    summary_rows.append([sheet_name, status, status_icon])

ux.table(summary_rows, headers=["Sheet", "Status", "Result"])

if validation_passed:
    ux.success("\n✓ All sheets validated successfully")
    step.log("Configuration validation completed: All sheets passed")
else:
    ux.error("\n✗ Some sheets failed validation")
    step.log("Configuration validation completed with errors", level="ERROR")

## 4) Review Validation Details

In [ ]:
# Display detailed validation results for sheets with errors or warnings
ux.subheader("Detailed Validation Results")

has_errors = False
has_warnings = False

for sheet_name, result in validation_data.items():
    errors = result.get('errors', [])
    warnings = result.get('warnings', [])
    
    if errors or warnings:
        ux.info(f"\n{sheet_name}:")
        
        if errors:
            has_errors = True
            ux.error(f"  Errors ({len(errors)}):")
            for error in errors:
                ux.error(f"    - {error}")
        
        if warnings:
            has_warnings = True
            ux.warning(f"  Warnings ({len(warnings)}):")
            for warning in warnings:
                ux.warning(f"    - {warning}")

if not has_errors and not has_warnings:
    ux.success("✓ No errors or warnings found")
elif has_errors:
    ux.error("\n✗ Configuration file has validation errors")
    ux.info("Please fix the errors and try again")
    raise ValueError("Configuration validation failed with errors")
elif has_warnings:
    ux.warning("\n⚠ Configuration file has warnings")
    ux.info("You may proceed, but review the warnings carefully")
    warning_count = sum(len(r.get('warnings', [])) for r in validation_data.values())
    step.log(f"Configuration has {warning_count} warning(s)", level="WARNING")

## 5) Preview Configuration Contents

In [ ]:
# Display key configuration data for review
ux.header("Configuration Contents Preview")

# Display Metadata
if 'Metadata' in config_data:
    ux.subheader("Metadata")
    metadata = config_data['Metadata']
    metadata_rows = [[k, v] for k, v in metadata.items() if not k.startswith('_')]
    ux.table(metadata_rows, headers=["Key", "Value"])

# Display Databases count
if 'Databases' in config_data:
    databases = config_data['Databases']
    ux.info(f"\n✓ Databases: {len(databases)} configured")

# Display EDM DB Upgrade count (same as databases, one upgrade per database)
if 'Databases' in config_data and 'Metadata' in config_data:
    edm_version = config_data['Metadata'].get('EDM Data Version')
    if edm_version:
        # Extract major version (e.g., "22.0.0" -> "22")
        target_version = edm_version.split('.')[0] if '.' in edm_version else edm_version
        ux.info(f"✓ EDM DB Upgrades: {len(databases)} jobs to upgrade to version {target_version}")

# Display Portfolios count
if 'Portfolios' in config_data:
    portfolios = config_data['Portfolios']
    base_portfolios = get_base_portfolios(portfolios)
    ux.info(f"✓ Base Portfolios: {len(base_portfolios)} configured")

# Display Reinsurance Treaties Count (as unique treaty-EDM job combinations)
if 'Reinsurance Treaties' in config_data and 'Analysis Table' in config_data:
    treaties = config_data['Reinsurance Treaties']
    analyses = config_data['Analysis Table']

    # Build a set of valid treaty names for validation
    valid_treaty_names = {t.get('Treaty Name') for t in treaties if t.get('Treaty Name')}

    # Collect unique treaty-EDM combinations from Analysis Table
    treaty_edm_combinations = set()
    treaty_columns = ['Reinsurance Treaty 1', 'Reinsurance Treaty 2', 'Reinsurance Treaty 3',
                      'Reinsurance Treaty 4', 'Reinsurance Treaty 5']

    for analysis in analyses:
        edm = analysis.get('Database')
        if not edm:
            continue
        for col in treaty_columns:
            treaty_name = analysis.get(col)
            if treaty_name and treaty_name in valid_treaty_names:
                treaty_edm_combinations.add((treaty_name, edm))

    ux.info(f"✓ Reinsurance Treaties: {len(treaties)} defined, {len(treaty_edm_combinations)} treaty-EDM jobs to create")

# Display GeoHaz count (base portfolios with geocode version)
if 'Portfolios' in config_data and 'Metadata' in config_data:
    geocode_version = config_data['Metadata'].get('Geocode Version')
    if geocode_version and base_portfolios:
        ux.info(f"✓ GeoHaz Jobs: {len(base_portfolios)} jobs using geocode version {geocode_version}")

# Display Portfolio Mapping count (base portfolios only)
if 'Portfolios' in config_data:
    base_portfolios = get_base_portfolios(portfolios)
    ux.info(f"✓ Portfolio Mapping Jobs: {len(base_portfolios)} base portfolios to map")

# Display Analysis count
if 'Analysis Table' in config_data:
    analyses = config_data['Analysis Table']
    ux.info(f"✓ Analysis Jobs: {len(analyses)} configured")

# Display Grouping counts (separated by type)
if 'Groupings' in config_data:
    groupings = config_data['Groupings']
    analysis_only_groups, rollup_groups = classify_groupings(config_data)
    ux.info(f"✓ Grouping Jobs (analysis-only): {len(analysis_only_groups)} configured")
    if rollup_groups:
        ux.info(f"✓ Grouping Rollup Jobs (groups of groups): {len(rollup_groups)} configured")

# Display Export to RDM job count (1 job that exports all analyses and groups)
if 'Metadata' in config_data and 'Analysis Table' in config_data and 'Groupings' in config_data:
    rdm_name = config_data['Metadata'].get('Export RDM Name')
    if rdm_name:
        analyses = config_data['Analysis Table']
        groupings = config_data['Groupings']
        analysis_names = [a.get('Analysis Name') for a in analyses if a.get('Analysis Name')]
        group_names = [g.get('Group_Name') for g in groupings if g.get('Group_Name')]
        total_exports = len(analysis_names) + len(group_names)
        ux.info(f"✓ Export to RDM: 1 job to export {len(analysis_names)} analyses and {len(group_names)} groups to '{rdm_name}'")

# Display Moody's Reference Data
if "Moody's Reference Data" in config_data:
    ux.subheader("\nMoody's Reference Data")
    ref_data = config_data["Moody's Reference Data"]
    
    if 'Model Profiles' in ref_data:
        ux.info(f"✓ Model Profiles: {len(ref_data['Model Profiles'])} available")
    if 'Output Profiles' in ref_data:
        ux.info(f"✓ Output Profiles: {len(ref_data['Output Profiles'])} available")
    if 'Event Rate Schemes' in ref_data:
        ux.info(f"✓ Event Rate Schemes: {len(ref_data['Event Rate Schemes'])} available")

## 6) Load Configuration to Database

In [ ]:
# Confirm before loading to database
ux.header("Load Configuration to Database")

if has_warnings:
    ux.warning("⚠ Configuration has warnings - proceed with caution")

# Ask for confirmation
proceed = ux.yes_no("Load this configuration to the database?")

if not proceed:
    ux.info("Configuration loading cancelled by user")
    step.log("User cancelled configuration loading")
    raise SystemExit("User cancelled configuration loading")

# Load configuration to database
ux.info("\nLoading configuration to database...")

# Get cycle ID
cycle_result = execute_query(
    "SELECT id FROM irp_cycle WHERE cycle_name = %s",
    (context.cycle_name,)
)

if cycle_result.empty:
    raise ValueError(f"Cycle not found: {context.cycle_name}")

cycle_id = int(cycle_result.iloc[0]['id'])  # Convert numpy.int64 to Python int

# Load configuration (this validates, deletes old configs, and loads new one)
# Wrap in try-except to properly handle ConfigurationError and mark step as failed
from helpers.configuration import ConfigurationError

try:
    config_id = load_configuration_file(cycle_id, str(config_path))
except ConfigurationError as e:
    error_msg = str(e)
    ux.error(f"\n✗ Configuration loading failed")
    ux.error(f"  {error_msg}")
    step.fail(error_msg)
    raise

ux.success(f"\n✓ Configuration loaded successfully")
ux.info(f"Configuration ID: {config_id}")

# Log configuration loading
step.log(f"Configuration loaded to database with ID: {config_id}")
step.log(f"Databases: {len(config_data.get('Databases', []))}, Portfolios: {len(config_data.get('Portfolios', []))}, Analyses: {len(config_data.get('Analysis Table', []))}")

# Retrieve and display loaded configuration info
loaded_config = read_configuration(config_id)

config_info = [
    ["Configuration ID", loaded_config['id']],
    ["Cycle ID", loaded_config['cycle_id']],
    ["Status", loaded_config['status']],
    ["Created", loaded_config['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(config_info, headers=["Property", "Value"])

## 7) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Complete the step with output data
step.complete({
    'configuration_id': config_id,
    'configuration_file': str(config_path),
    'validation_passed': validation_passed,
    'has_warnings': has_warnings,
    'has_errors': has_errors,
    'databases_count': len(config_data.get('Databases', [])),
    'portfolios_count': len(config_data.get('Portfolios', [])),
    'analyses_count': len(config_data.get('Analysis Table', []))
})

ux.success("\n" + "="*60)
ux.success("✓ CONFIGURATION LOADED SUCCESSFULLY")
ux.success("="*60)
ux.info("\nYou can now proceed to Stage 02: Data Extraction")